In [4]:
from os import environ as ENV
from time import time as now
import json
import qbraid

import pandas as pd
from tqdm.notebook import tqdm

import sys
sys.path.append(ENV['PWD'])

from should_be_stdlib import *

In [5]:
# number of shots to use
SHOTS = 1000

# Load circuits

In [22]:
ang_circuits = pd.read_excel(ENV['PWD'] + '/data/circuits_quantum_fidelity_angle_embed.xlsx', index_col=0)
amp_circuits = pd.read_excel(ENV['PWD'] + '/data/circuits_quantum_fidelity_amp_embed.xlsx', index_col=0)
amp_qft_circuits = pd.read_excel(ENV['PWD'] + '/data/circuits_quantum_fidelity_amp_embed_qft.xlsx', index_col=0)
[x.shape for x in [ang_circuits, amp_circuits, amp_qft_circuits]]

[(2926, 3), (2926, 3), (2926, 3)]

In [23]:
# transpile circuits ahead of time
# if IonQ-qiskit is installed, the ionq job submission expects qiskit circuits instead of the native ionq format!!
# but qiskit transpiler is much faster so it's fine
import qbraid
amp_qft_circuits  = [qbraid.transpile(c, 'qiskit') for c in tqdm(amp_qft_circuits['qasm2'])]
amp_circuits      = [qbraid.transpile(c, 'qiskit') for c in tqdm(amp_circuits['qasm2'])]
ang_circuits      = [qbraid.transpile(c, 'qiskit') for c in tqdm(ang_circuits['qasm2'])]

  0%|          | 0/2926 [00:00<?, ?it/s]

  0%|          | 0/2926 [00:00<?, ?it/s]

  0%|          | 0/2926 [00:00<?, ?it/s]

# IonQ

## Connect to IonQ API

In [9]:
from provider_ionq import IONQ
ionq = IONQ(
    ionq_api_key := open(ENV['HOME'] + "/work/api-keys/IONQ_API_KEY").readline().strip()
)
ionq.printqpus()

In [10]:
ionq_qpu = ionq.provider.get_device('qpu.forte-enterprise-1')
ionq_qpu.status()

<DeviceStatus.ONLINE: 'online'>

# Run circuits

In principle, one could run these circuits. However, the estimated cost of IonQ Forte is very high, so we chose not to run them to save money.

In [11]:
import warnings
warnings.filterwarnings('ignore', message='No gate definition for.*') # hide transpiler warnings

In [12]:
# this submits a preflight job, no costs are incurred
def test_run(*args, **kwargs):
    kwargs['preflight'] = True
    ionq_job = ionq_qpu.run(*args, **kwargs)
    print(f'Estimating cost of {kwargs["name"]}')
    ionq_job.wait_for_final_state()
    metadata = ionq_job.metadata()
    cost_usd = metadata["cost_usd"]
    predicted_execution_time = metadata["predicted_execution_time"]
    print(f'${cost_usd}, {predicted_execution_time / 1000} seconds')

In [18]:
def verify_run():
    ans = input('You must type "ionq" to run this because it may cost a lot of money!!! >>> ')
    if ans != 'ionq':
        raise ValueError('By not typing "ionq", you did not run this code')
        return False
    else:
        ans = input('If you are really sure you want to run this code, type "ionq" to proceed >>> ')
        if ans != 'ionq':
            raise ValueError('By not typing "ionq", you did not run this code')
            return False
        else:
            return True

In [19]:
# test run our jobs
if verify_run():
    test_run(amp_qft_circuits[0], preflight=True, shots=SHOTS, name="Amp QFT",           error_mitigation={'debias': True})
    test_run(amp_qft_circuits[0], preflight=True, shots=SHOTS, name="Amp QFT no QEM",    error_mitigation={'debias': False})
    test_run(amp_circuits[0],     preflight=True, shots=SHOTS, name="Amp no QFT",        error_mitigation={'debias': True})
    test_run(amp_circuits[0],     preflight=True, shots=SHOTS, name="Amp no QFT no QEM", error_mitigation={'debias': False})
    test_run(ang_circuits[0],     preflight=True, shots=SHOTS, name="Angle",             error_mitigation={'debias': True})
    test_run(ang_circuits[0],     preflight=True, shots=SHOTS, name="Angle no QEM",      error_mitigation={'debias': False})

You must type "ionq" to run this because it may cost a lot of $$money$$ !!! >>>  no


ValueError: By not typing "ionq", you did not run this code

In [25]:
# run for real
if verify_run():
    ionq_qpu.run(amp_qft_circuits[0], shots=SHOTS, name="REAL Amp QFT",           error_mitigation={'debias': True})
    ionq_qpu.run(amp_qft_circuits[0], shots=SHOTS, name="REAL Amp QFT no QEM",    error_mitigation={'debias': False})
    ionq_qpu.run(amp_circuits[0],     shots=SHOTS, name="REAL Amp no QFT",        error_mitigation={'debias': True})
    ionq_qpu.run(amp_circuits[0],     shots=SHOTS, name="REAL Amp no QFT no QEM", error_mitigation={'debias': False})
    ionq_qpu.run(ang_circuits[0],     shots=SHOTS, name="REAL Angle",             error_mitigation={'debias': True})
    ionq_qpu.run(ang_circuits[0],     shots=SHOTS, name="REAL Angle no QEM",      error_mitigation={'debias': False})

You must type "ionq" to run this because it may cost a lot of $$money$$ !!! >>>  ionq
If you really are sure you want to run this code, type "ionq" to proceed >>>  ionq
